<div >
<img src = "../../../ans_banner_1920x200.png" />
</div>

# Introducción al Análisis de Clusters

Este *cuaderno* presenta una introducción al análisis de clusters. El  análisis de clusters aglomera un conjunto de técnicas y algoritmos que tienen por objetivo buscar observaciones o variables similares y agruparlos en grupos o clusters. El objetivo del *cuaderno* es que aprenda a que nos referimos cuando hablamos de clusters, qué implica buscar grupos similares, qué se entiende por similaridad, y cómo la medimos.

**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar  libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.


## ¿Qué es el análisis de clusters? 

El análisis de clusters es una de las principales aplicaciones de los algoritmos de aprendizaje no supervisado. Este tipo de análisis se utiliza para juntar observaciones similares en grupos, por ejemplo:

- Segmentar a clientes por preferencias o sensibilidad a cambios en los precios.
- Encontrar usuarios de una aplicación de música por géneros o bandas que prefieren.
- Agrupar a votantes de acuerdo con los temas que les importa al momento de votar.
- Dividir un corpus de documentos en temas.

El gráfico siguiente ilustra el resultado de un análisis de clusters basado en dos variables:


<div style="max-width:500px">
<img src = "figs/plot_clustering_notebook.png" />
</div>

Se puede apreciar que las observaciones se pueden separar claramente en 3 grupos o clusters, donde las observaciones son lo mas similares dentro de los clusters y  disimilares entre ellos.

Las etapas del análisis de clusters podemos resumirlas de la siguiente forma:

1. Iniciamos con una matriz de datos

    \begin{align}
X_{n\times k}=\left(\begin{array}{cccc}
x_{11} &  & \dots & x_{1k}\\
\\
\vdots &  & x_{ik} & \vdots\\
\\
x_{n1} &  & \dots & x_{nk} 
\end{array}\right)
    \end{align}

2. Calculamos la matriz de distancia o disimilitud

\begin{align}
D_{n\times n}=\left(\begin{array}{ccccc}
d_{11} &  & \dots &  & d_{1n}\\
 & \ddots\\
\vdots &  & d_{jj} &  & \vdots\\
 &  &  & \ddots\\
d_{n1} &  & \dots &  & d_{nn}
\end{array}\right)
\end{align}


3. Aplicamos el algoritmo de clustering. Existen varios tipos de estos algoritmos que en términos generales podemos dividirlos en cuatro categorías:

    - **Algoritmos basados en centroides**: cada cluster está representado por un centroide. Los clusters se construyen en función de la distancia al centroide del grupo.
    
    - **Algoritmos basados en conectividad**: los clusters se definen agrupando los datos respecto al vecino más cercano. La idea es que los puntos de datos cercanos estén más relacionados que otros puntos más lejanos. El aspecto clave es que un grupo contiene otros grupos. Debido a esta estructura, los clusters representan una jerarquía.

    - **Algoritmos basados en densidades**: los clusters aquí se definen por áreas de densidad concentrada. Este método comienza buscando áreas de puntos de datos densos y asigna esas áreas a los mismos clusters.
    
    - **Algoritmos basados en distribuciones**: este método asume que cada cluster pertenece a una distribución normal. La idea es que los puntos de datos se dividan en función a la probabilidad de pertenecer a la misma distribución normal. Es similar a la agrupación en clusters basada en densidades y en centroides, excepto que genera los clusters asumiendo un proceso generador de datos gaussianos y utiliza estas probabilidades para asignar a los clusters en lugar de un centroide. 


Las preguntas naturales que surgen son: (1) ¿Cómo se mide disimilitud o distancia? y (2) ¿Qué algoritmo de clustering puedo aplicar para identificar estos grupos? En este *cuaderno* responderemos la primer pregunta y en los *cuadernos* subsiguientes la segunda. 


## Medidas de distancia o disimilitud.

Para poder agrupar observaciones en clusters es necesario primero calcular la  distancia o disimilitud entre ellas, para  luego armar la matriz de distancias. La distancia expresa la proximidad o la similitud  que existe entre individuos o variables.  

Existen ciertas guías para escoger una adecuada medida de distancia o disimilitud, sin embargo, existe un buen grado de subjetividad en el momento de escogerlas. Por esto, primero es necesario estudiar las medidas más utilizadas antes de explorar los algoritmos de clustering.

### Medidas de distancia entre observaciones

Matemáticamente se da el nombre de medida de distancia entre dos observaciones $i$ e $i'$, $d_{ii'}$ a toda medida que verifique todos los siguientes axiomas:

1. $d(x_{i},x_{i'})\geq 0$ y $d(x_{i'},x_{i})=0$ 
2. $d(x_{i},x_{i'})=d(x_{i'},x_{i})$
3. $d(x_{i},x_{i'})\leq d(x_{i},x_{i''})+d(x_{i'},x_{i''})$


donde  $x_{i}$ y $x_{i'}$ son las variables o atributos de estas observaciones. Entre las medidas de distancia, la más conocida es la  distancia euclidiana que se utiliza para variables cuantitativas. Sin embargo, existen otras medidas de distancia que se utilizarán dependiendo el tipo o la naturaleza de las variables, por lo que los resultados del algoritmo serán sensibles a la elección de la medida.

A continuación, exploraremos las medidas de distancia más utilizadas por tipo de datos:


#### Distancia para variables cuantitativas. 

Las variables cuantitativas son aquellas en las que su dominio es un continuo de números reales. Para este tipo de datos la distancia, la mas utilizada suele ser la distancia euclidiana. Para dos números reales esta dada por:


\begin{align}
d_{ii'}=d(x_i,x_{i'})= \sqrt{(x_i - x_{i'})^2}
\end{align}

es decir, la raíz cuadrada del cuadrado de las diferencias. 

Ilustremos el cálculo de esta distancia en `Python` utilizando datos de 5 individuos que contienen información sobre su Sexo, Edad, Educación e Ingresos. Primero hay que notar que en esta base hay:
- Dos variables cuantitativas: Edad e Ingresos.
- Una variable categórica: Sexo.
- Una variable ordinal: Educación.


In [1]:
# Cargamos las librerías a utilizar
import pandas as pd
import math

# Cargamos y visualizamos los datos
datos = pd.read_csv('data/data_eph.csv')
datos.head()

,Sexo,Edad,Educacion,Ingresos
0,Hombre,66,Secundario,52500
1,Hombre,70,Secundario,126400
2,Mujer,23,Primario,32400
3,Mujer,20,Universitario,74000
4,Hombre,22,Terciario,39000


La distancia de la Edad de la primera observación respecto a la segunda será:
    
$d_{01}= \sqrt{(66-70)^2}$ 

In [2]:
math.sqrt((66-70)**2)

4.0

Otros tipos de distancias para variables cuantitativas son:

   - Distancia de Manhattan: $|x_i - x_{i'}|$
    
   - Distancia de Minkowski: $(|x_i - x_{i'}|^p)^{\frac{1}{p}}$
    
   - Distancia de Chebychev: $Max|x_i - x_{i'}|$

#### Distancia para variables categóricas ordenadas.

Las variables categóricas ordenadas  se representan como un continuo de enteros, cuyo dominio es un conjunto ordenado, como es el caso de Educación en el ejemplo anterior:
\begin{align}
\left(\begin{array}{c}
Primario\\
Secundario\\
Terciario\\
Universitario
\end{array}\right)\rightarrow\left(\begin{array}{c}
1\\
2\\
3\\
4
\end{array}\right)
\end{align}

En este caso, primero reemplazamos sus valores originales con:

\begin{align}
\frac{i-1/2}{M} \,\,\, i=1,...,M
\end{align}


donde $M$ es el número de categorias.  En `Python` esta transformación se puede lograr haciendo:


In [3]:
Edu = datos.loc[:,['Educacion']]
Edu

,Educacion
0,Secundario
1,Secundario
2,Primario
3,Universitario
4,Terciario


In [4]:
educacion = ["Primario", "Secundario", "Terciario", "Universitario"]
orden = [1, 2, 3, 4]

Edu = Edu.replace(educacion, orden)
Edu

,Educacion
0,2
1,2
2,1
3,4
4,3


In [5]:
Edu = (Edu - 0.5)/len(educacion)
Edu

,Educacion
0,0.375
1,0.375
2,0.125
3,0.875
4,0.625


Luego utilizamos las medidas para variables cuantitativas descritas anteriormente.

#### Distancia para variables categóricas no ordenadas.

Las variables categóricas  no ordenadas son aquellas que tienen dos o mas categorías, pero no existe un ordenamiento intrínseco de las mismas, como es el caso de educación en el ejemplo anterior.

Si la variable es binaria, se codifica a una de las categorías con $0$ y a la otra con $1$, esto se lo suele denominar como *one hot encoding*. Este procedimiento hace que las variables categóricas se vuelvan numéricas y podamos medir distancia. La medida de distancia más utilizada en estos casos se conoce como distancia de coincidencia simple:


\begin{align}
d_{ii'}=d(x_i,x_{i'})= I(x_i \neq x_{i'})
\end{align}

donde $I(.)$ es la función indicadora que toma valor 1 cuando las variables no coinciden, y 0 en caso contrario. En el ejemplo anterior, si codificamos Hombre como 1 y Mujer como 0 tendríamos:

\begin{align}
d_{01}= 0
\end{align}

y 

\begin{align}
d_{02}= 1
\end{align}

La distancia de la primera observación a la segunda es 0 puesto que ambos son hombres, mientras que de la primera a la tercera es 1 ya que los sexos no coinciden.

Cuando las variables tienen más de dos categorías, se dicotomiza adecuadamente cada una de ellas (con ceros y unos) y luego se utilizan las medidas que definimos anteriormente. Por ejemplo, si una variable contiene el estado civil: Soltero, Casado, Separado. Entonces crearíamos 3 variables dicótomas, una para Soltero, Casado, y Separado:
\begin{align}
\left(\begin{array}{c}
Soltero\\
Casado\\
Soltero\\
Separado\\
Casado
\end{array}\right)\rightarrow\left(\begin{array}{ccc}
Soltero & Casado & Separado\\
1 & 0 & 0\\
0 & 1 & 0\\
1 & 0 & 0\\
0 & 0 & 1\\
0 & 1 & 0
\end{array}\right)
\end{align}


### Medidas de  distancia entre varias variables.

En la práctica, a menudo nos encontramos que cada observación tiene más de una  variable o atributo $j=1,2,\cdots,p$. Por ende, para estimar la matriz de distancia debemos saber cómo combinar las distancias entre observaciones para múltiples variables.  La forma mas común cuando las variables son cuantitativas es utilizar la distancia euclidiana, entonces la distancia entre las observaciones $i$ e $i'$ cuando tienen $p$ atributos es:

\begin{align}
d(x_{i},x_{i'})= \sqrt{\sum_{j=1}^p (x_{ij}- x_{i'j})^2}
\end{align}

Extensiones de las medidas exploradas anteriormente también son posibles, y uno debe ser cuidadoso porque los resultados van a ser sensibles a esta la elección.

Por ejemplo, la distancia entre el primer individuo y el segundo individuos para los atributos cuantitativos de los datos anteriores sería:

In [6]:
datos_cuant=datos.loc[:,['Edad','Ingresos']]
datos_cuant

,Edad,Ingresos
0,66,52500
1,70,126400
2,23,32400
3,20,74000
4,22,39000


$d_{01}= \sqrt{(66-70)^2+(52500-126400)^2}$ 

In [7]:
math.sqrt((66-70)**2+(52500-126400)**2)

73900.0001082544

Otra medida que se utiliza cuando tenemos multiples variables dicótomas es la distancia de Jaccard, ésta es:

\begin{align}
d_{ii'}=d(x_i,x_{i'})= 1 - \frac{\sum_k I(x_{1k}\, y \,x_{i'k})}{\sum_k I(x_{1k}\, o\, x_{i'k})}
\end{align}


La distancia euclidiana en el caso anterior combina las $p$ distancias dándole el mismo peso a todos los atributos, sin embargo, se pude definir también como un promedio ponderado y combinar distintas medidas de distancia. Esto quedaría como una combinación convexa:

\begin{align}
d_{ii'}=(x_i, x_{i'})=\sum_{j=1}^p\omega_j\cdot d_j(x_{ij}, x_{i'j});\ \sum_{j=1}^p\omega_j=1
\end{align}

en donde $\omega_j$ es el peso asignado al atributo j-ésimo; y se elegirá subjetivamente dependiendo del problema. $d_j(x_{ij}, x_{i'j})$ es una de las medidas de distancia que describimos anteriormente y se elegirá dependiendo las características de la variable o atributo, y del problema bajo estudio.
 
Es importante recalcar que pesos iguales no necesariamente conlleva a que todos los atributos tengan la misma influencia, por ejemplo $\omega_j=1\ \forall j$. La influencia del atributo j-ésimo $X_j$ en la disimilaridad entre dos objetos depende en su contribución relativa al promedio de disimilaredes entre todos los puntos de un conjunto de datos:

\begin{align}
\bar{d}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N d(x_i, x_{i'})=\sum_{j=1}^p\omega_j \cdot \bar{d_j}
\end{align}


Con $\bar{d_j}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N d_j(x_{ij}, x_{i'j})$ siendo la disimilaridad promedio del atributo $j$. Por consiguiente, la influencia relativa de la variable j-ésima es $\omega_j \cdot \bar{d_j}$ y definiendo $\omega_j \sim 1/\bar{d_j}$ se consigue que todos los atributos aporten lo mismo a la disimilaridad.

Por ejemplo, si se tienen $p$ variables cuantitativas y se define el error al cuadrado como medida de distancia para cada coordinada, obtenemos la distancia euclidiana al cuadrado ponderada por pesos:

\begin{align}
d_I(x_i, x_{i'})=\sum_{j=1}^p\omega_j\cdot (x_{ij}-x_{i'j})^2
\end{align}

Note que $\bar{d_j}=\frac{1}{N^2}\sum_{i=1}^N\sum_{i'=1}^N (x_{ij}-x_{i'j})^2=2\cdot var_j$ en donde $var_j$ es la varianza muestral de $x_j$. Luego, la importancia relativa de cada variable es proporcional a la varianza de esta variable en el conjunto de datos. 


Finalmente, y quizás el método más popular para medir distancia o disimilaridad es usar la distancia de Gower. Esta permite medir la distancia entre dos variables numéricas y no numéricas. El proceso es el siguiente:

1. Definir un score de disimilaridad para cada dos observaciones $i$ e $i'$:

    - Si la variable es cuantitativa:
        \begin{align}
               s_j(x_{ij},x_{i'j})= 1-\frac{|x_{ij}-x_{i'j}|}{n}
        \end{align}

    - Si la variable es categórica: 
         
        \begin{align}
               s_j(x_{ij},x_{i'j})=\begin{cases}
                1 & x_{ij}=x_{i'j}\\
                0 & x_{ij}\neq x_{i'j}
               \end{cases}
        \end{align}
         
             
Note que este score será: $s_{ii'j}=0$ si $x_{ij}$ y $x_{i'j}$ son disimilares, y $0<s_{ii'j}\leq1$ , si no son disimilares.


2. Definir $\delta_{ii'j}=0$ si $x_{ij}$ y $x_{i'j}$  no pueden compararse por datos faltantes, y 1 en caso contrario.

3. Agregar el score de disimilaridad para $i$ e $i'$:
\begin{align}
            s_j(x_{ij},x_{i'j}) = \frac{\sum_{j=1}^p    s_j(x_{ij},x_{i'j})}{\sum_{j=1}^p\delta_{ii'j}}
\end{align}

Al igual que en el caso anterior, podemos utilizar pesos para reflejar la importancia relativa de las variables:
\begin{align}
         s_j(x_{ij},x_{i'j}) = \frac{\sum_{j=1}^p   \omega_j s_j(x_{ij},x_{i'j})}{\sum_{j=1}^p \omega_j\delta_{ii'j}}
        \end{align}

#### Correlación entre variables

Dependiendo del tipo de datos y la pregunta que se busque responder, podemos estar interesados no en si las observaciones o variables están cerca una de otras, sino en si sus características están altamente correlacionadas, aunque los valores observados estén muy distantes en términos de distancia euclidiana.

Si queremos identificar clusters de observaciones o variables con los mismos perfiles generales independientemente de sus magnitudes, entonces deberemos utilizar medidas de disimilitud basados en las correlaciones. Por ejemplo, en marketing, podríamos querer identificar clusters de clientes con la misma preferencia en términos de artículos, independientemente del volumen de compra de los artículos.

Los coeficientes de correlación de Pearson y de Spearman son algunos de los  más utilizados.

##### Coeficiente de correlación de Pearson

El coeficiente de correlación de Pearson se define como:

\begin{align}
\rho(x_i, x_{i'})=\frac{\sum_k (x_{ik}-\bar{x_i})(x_{i'k}-\bar{x_{i'}})}{\sqrt{\sum_k (x_{ik}-\bar{x_i})^2\sum_j(x_{i'k}-\bar{x_{i'}}^2)}}
\end{align}

en donde $\bar{x_i}=\sum_k x_{ik}/p$. Note que el promedio es sobre las variables y no sobre las observaciones. Si se estandarizan antes las observaciones, entonces 

\begin{align}
\sum_k (x_{ik}-x_{i'k})^2 \propto 2(1-\rho(x_i, x_{i'}))
\end{align}

Por esto los clusters basados en la correlación son equivalentes a los clusters basados en la distancia al cuadrado.

Este coeficiente toma valores entre -1 y 1. Si $\rho(x_i, x_{i'})=0$  implica de que no existe una correlación lineal entre $x_i$ y $ x_{i'}$, mientras que si es -1 hay una correlación lineal negativa, y cuando es 1, una correlación lineal positiva.

Es importante notar que este coeficiente es invariante a las unidades de $x_i$, $x_{i'}$, pero es sensible a los valores atípicos. Esto se puede mitigar utilizando el coeficiente correlación de Spearman en lugar de la correlación de Pearson.

En `Python` este se calcula como:

In [8]:
datos.corr() # Por defecto hace la correlación de Pearson

,Edad,Ingresos
Edad,1.000000,0.611735
Ingresos,0.611735,1.000000


Note que cuando aplicamos el método `.corr()` sobre un `dataframe`, este automáticamente tiene sólo en cuenta las variables numéricas.

##### Coeficiente de correlación de Spearman

El coeficiente de correlación de Spearman es una medida no paramétrica de correlación de rangos y se define como:

\begin{align}
r_s(R(x_i), R(x_{i'}))=\frac{Cov(R(x_i), R(x_{i'}))}{Var(R(x_i))Var(R(x_{i'}))}
\end{align}

en donde $R(.)$ denota el rango de las variables. Este coeficeinte también toma valores entre -1 y 1, y coincide con el coeficiente de Pearson si se aplica a los rangos de esas variables. Sin embargo, mientras que la correlación de Pearson mide  relaciones lineales, la correlación de Spearman evalúa las relaciones monótonas (ya sean lineales o no). Hay que notar también, que este coeficiente es robusto a valores atípicos, porque se calcula sobre el rango de las variables, que no se ve afectado por la presencia de valores atípicos.

En `Python` este se calcula como:

In [9]:
datos.corr("spearman")

,Edad,Ingresos
Edad,1.0,0.3
Ingresos,0.3,1.0


Otra opción que permite el método `.corr()` es el  coeficiente de correlación de rango de Kendall

In [10]:
datos.corr("kendall")

,Edad,Ingresos
Edad,1.0,0.2
Ingresos,0.2,1.0



##  Matriz de distancia o disimilitud.

La matriz de distancia recoge las distancias anteriormente calculadas entre los objetos. Por lo tanto, si estamos midiendo la distancia entre $n$ observaciones, la matriz resultante será de tambaño $n\times n$  y cada elemento $d_{ii'}$ corresponde a la distancia entre  $i$ e $i'$:

\begin{align}
D_{n\times n}=\left(\begin{array}{ccccc}
d_{11} &  & \dots &  & d_{1n}\\
 & \ddots\\
\vdots &  & d_{jj} &  & \vdots\\
 &  &  & \ddots\\
d_{n1} &  & \dots &  & d_{nn}
\end{array}\right)
    \end{align}
    

La mayoría de los algoritmos suponen que la matriz de disimilitud debe ser no negativa y con ceros en la diagonal (es decir que la distancia de un objeto a si mismo es 0).

\begin{align}
D_{n\times n}=\left(\begin{array}{ccccc}
0 &  & \dots &  & d_{1n}\\
 & \ddots\\
\vdots &  & 0 &  & \vdots\\
 &  &  & \ddots\\
d_{n1} &  & \dots &  & 0
\end{array}\right)
    \end{align}
    

Esta matriz se puede transformar para que refleje cercanía-similaridad  o distancia-disimilitud simplemente aplicando una transformación monótona creciente (o decreciente). Adicionalmente es usual suponer que la matriz es simétrica, por lo qué si originalmente esta no lo es, se puede aplicar la transformación $(D+D^T)/2$.

Cabe notar que en algunas aplicaciones como en las ciencias sociales, esta matriz es a veces remplazada por una que contiene entradas subjetivas que reflejan afinidad. Por ejemplo, en experimentos de ciencias sociales, a los participantes se les pide que juzguen qué tan similares son dos objetos. En estos casos hay que ser cuidadoso, pues este tipo de medidas rara vez constituyen una medida de distancia pues no satisfacen la desigualdad triangular $d_{ii'}\leq d_{ik} + d_{i'k}$, para todo $k\in\{1, \cdots, N\}$. Por tal motivo, algunos algoritmos no permiten usar este tipo de matrices.

En `Python` para calcular la matriz necesitamos de `scipy` `distance_matrix`, que por defecto calcula la distancia de Minkowski. Sin embargo, si elegimos $p=2$ esta distancia coincide con la euclidiana:

In [11]:
import numpy as np
from scipy.spatial import distance_matrix

# Seleccionamos solo las columnas que sean numéricas
datos_numericos = datos.select_dtypes(np.number)

# Creamos la matriz de distancias escogiendo p = 2, el cual convierte 
# la distancia Minkowski en la distancia euclidiana
dist = distance_matrix(datos_numericos, datos_numericos, p = 2)

# Para visualizar mejor la matriz de distancia la convertimos en un dataframe
pd.DataFrame(dist)

,0,1,2,3,4
0,0.000000,73900.000108,20100.045995,21500.049209,13500.071704
1,73900.000108,0.000000,94000.011750,52400.023855,87400.013181
2,20100.045995,94000.011750,0.000000,41600.000108,6600.000076
3,21500.049209,52400.023855,41600.000108,0.000000,35000.000057
4,13500.071704,87400.013181,6600.000076,35000.000057,0.000000


#### Matrices de distancia o disimilitud con datos estandarizados.

Un problema de la distancia euclidiana, como medida de similaridad, es su dependencia de las diferentes escalas en que estén medidas las variables. Escalas y rangos de variación diferentes pueden afectar al análisis de clusters. Este problema se soluciona si en vez de calcular la distancia euclidiana con las unidades originales, se calcula con puntuaciones estandarizadas. 

Para ver cómo cambia el cálculo anterior estandaricemos los datos:


In [12]:
mu = datos_numericos.mean()
sigma = datos_numericos.std()
Z = (datos_numericos - mu)/sigma
Z

,Edad,Ingresos
0,1.014146,-0.326115
1,1.171378,1.623714
2,-0.676097,-0.856447
3,-0.794021,0.241156
4,-0.715405,-0.682308


In [13]:
dist = distance_matrix(Z, Z, p = 2)
pd.DataFrame(dist)

,0,1,2,3,4
0,0.000000,1.956158,1.771489,1.895063,1.765849
1,1.956158,0.000000,3.092630,2.402969,2.979545
2,1.771489,3.092630,0.000000,1.103920,0.178520
3,1.895063,2.402969,1.103920,0.000000,0.926805
4,1.765849,2.979545,0.178520,0.926805,0.000000


Note que cuando usamos los datos sin estandarizar, la observación más cercana a la primera persona es la quinta. Sin embargo, una vez normalizamos nuestras variables, encontramos que la tercera observación es la más cercana a la primera persona. Esto sucede porque cuando no escalamos nuestras variables, aquellas como la edad aportan muy poco a la distancia en comparación a otras variables como ingreso que tienden a estar en unidades mayores. Esta diferencia será importante al momento de comparar las distancias entre las distintas observaciones y es una decisión subjetiva que deberá tenerse en cuenta.

#### Matrices de proximidad o disimilitud con datos mixtos.

Anteriormente mencionamos que cuando tenemos matrices con datos mixtos como los datos de ejemplo, se puede aplicar la distancia de Gower: 

In [14]:
datos

,Sexo,Edad,Educacion,Ingresos
0,Hombre,66,Secundario,52500
1,Hombre,70,Secundario,126400
2,Mujer,23,Primario,32400
3,Mujer,20,Universitario,74000
4,Hombre,22,Terciario,39000


En `Python` lo hacemos utilizando la libreria `gower`:

In [15]:
import gower 

dist_g = gower.gower_matrix(datos)
pd.DataFrame(dist_g)

,0,1,2,3,4
0,0.000000,0.216543,0.768457,0.787181,0.505904
1,0.216543,0.000000,0.985000,0.889362,0.722447
2,0.768457,0.985000,0.000000,0.375638,0.522553
3,0.787181,0.889362,0.375638,0.000000,0.603085
4,0.505904,0.722447,0.522553,0.603085,0.000000


Note que una vez tenemos en cuenta los diferentes tipos de variables, la distancia entre las diferentes personas de la muestra cambia. Por ejemplo, cuando solo tenemos la distancia euclidiana la persona 1 y la persona dos son bastantes disimiles pues la segunda tiene más del doble de ingresos que la primera. Sin embargo, cuando usamos la distancia de Gower podemos tener en cuenta otras variables como el sexo y la educación en las cuales ambos sujetos son idénticos, por lo que la distancia entre ambos no debería ser tan lejana.

## Análisis de clusters y otras formas de denominarlo.

Antes de finalizar este *cuaderno* es importante notar que en la literatura en español se suele encontrar el análisis de clusters con otras denominaciones como: segmentación de datos, taxonomía numérica, análisis tipológico, clasificación automática, entre otros. En este curso nos referiremos a análisis de clusters puesto que no sólo es la traducción directa del término *cluster analysis* con el que se lo suele encontrar en la literatura de aprendizaje no supervisado, sino también el nombre con el que se suele encontrar en la mayoría de los paquetes de software.

## Referencias


-  Hastie, T., Tibshirani, R., & Friedman, J. H. (2001). The elements of statistical learning: Data mining, inference, and prediction. New York: Springer.

- Kaufman, L. & Rousseeuw, P. (1990). Finding Groups in Data: An Introduction to Cluster Analysis, Wiley, New York.

- Tsai, K. T. (2021). Machine Learning for Knowledge Discovery with R: Methodologies for Modeling, Inference, and Prediction.

- Macnaughton Smith, P., Williams, W., Dale, M. & Mockett, L. (1965). Dissimilarity analysis: a new technique of hierarchical subdivision, Nature 202: 1034–1035.

- Pérez López, C. (2004). Técnicas de análisis multivariante de datos. Aplicaciones con SPSS, Madrid, Universidad Complutense de Madrid, 121-154.

- [Scikit-learn: Machine Learning in Python](https://scikit-learn.org/stable/auto_examples/cluster/plot_linkage_comparison.html#sphx-glr-auto-examples-cluster-plot-linkage-comparison-py), Pedregosa et al., JMLR 12, pp. 2825-2830, 2011. 

- Taddy, Matt; Taddy, Matt. Business Data Science: Combining Machine Learning and Economics to Optimize, Automate, and Accelerate Business Decisions (p. 202). McGraw-Hill Education.

